# Simple Gilmore Girls Chatbot

In [47]:
!pip install nltk

import nltk
import io
import random
import string # to process standard python strings
import warnings
import numpy as np
import re

from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords

nltk.download('popular', quiet=True) # for downloading popular packages
nltk.download('punkt') 
nltk.download('wordnet') 

warnings.filterwarnings('ignore')


[nltk_data] Downloading package punkt to /home/codespace/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### Brain of the chatbot is in 'chatbot.txt' file

The text in the 'chatbot.txt' is information from [Gilmore Girls](https://en.wikipedia.org/wiki/Gilmore_Girls) and [Gilmore Girls Wiki | Fandom](https://gilmoregirls.fandom.com/wiki/Gilmore_Girls).

In [48]:
f=open('chatbot.txt','r',errors = 'ignore')
raw=f.read()
raw = raw.lower()# converts to lowercase

### Tokenize

In [49]:
sent_tokens = nltk.sent_tokenize(raw)# converts to list of sentences 
word_tokens = nltk.word_tokenize(raw)# converts to list of words

### Key word matching - Greetings

In [50]:
GREETING_INPUTS = ("hello", "hi", "greetings", "sup", "what's up","hey",)
GREETING_RESPONSES = ["hi", "hey", "*nods*", "hi there", "hello", "I am glad! You are talking to me"]
def greeting(sentence):
 
    for word in sentence.split():
        if word.lower() in GREETING_INPUTS:
            return random.choice(GREETING_RESPONSES)

### Response

In [51]:
# Initialize
lemmatizer = WordNetLemmatizer()

def LemNormalize(text):
    clean_text = re.sub(r'[^a-zA-Z]', ' ', text).lower()
    # Tokenizie
    tokens = nltk.word_tokenize(clean_text)
    # Lemmatizing 
    lem_tokens = [lemmatizer.lemmatize(token) for token in tokens if token not in set(stopwords.words('english'))]
    return lem_tokens

TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english')


[nltk_data] Downloading package wordnet to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [52]:
def response(user_response):
    robo_response=''
    sent_tokens.append(user_response)
    TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english')
    tfidf = TfidfVec.fit_transform(sent_tokens)
    vals = cosine_similarity(tfidf[-1], tfidf)
    idx=vals.argsort()[0][-2]
    flat = vals.flatten()
    flat.sort()
    req_tfidf = flat[-2]
    if(req_tfidf==0):
        robo_response=robo_response+"I am sorry! I don't understand you"
        return robo_response
    else:
        robo_response = robo_response+sent_tokens[idx]
        return robo_response

### Conversations

In [54]:
flag=True
print("ROBO: My name is  Lorelai. I will answer yourquestions about Gilmore Girls. If you want to leave, type Bye!")
while(flag==True):
    user_response = input()
    user_response=user_response.lower()
    print(f"USER: {user_response}")  # This line echoes the user's input.
    if(user_response!='bye'):
        if(user_response=='thanks' or user_response=='thank you'):
            flag=False
            print("ROBO: You are welcome..")
        else:
            if(greeting(user_response)!=None):
                print("ROBO: "+greeting(user_response))
            else:
                print("ROBO: ",end="")
                print(response(user_response))
                sent_tokens.remove(user_response)
    else:
        flag=False
        print("ROBO: Bye! From the Gilmore Girls.")


ROBO: My name is  Lorelai. I will answer yourquestions about Gilmore Girls. If you want to leave, type Bye!


USER: hey
ROBO: I am glad! You are talking to me
USER: what is the gilmore girls?
ROBO: gilmore girls

gilmore girls (styled onscreen as gilmore girls) is an american comedy-drama television series created by amy sherman-palladino and starring lauren graham (lorelai gilmore) and alexis bledel (rory gilmore).
USER: daughter
ROBO: sookie gives birth to her daughter, martha.
USER: mom
ROBO: I am sorry! I don't understand you
USER: what is gilmore girls known for?
ROBO: gilmore girls is known for its fast-paced dialogue filled with pop-culture references.
USER: bye
ROBO: Bye! From the Gilmore Girls.
